In [ ]:
#Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

Using TensorFlow backend.


In [ ]:
#Defining paths of training and testing datasets
TRAIN_PATH = "C:/Users/Sagar Nath/Desktop/ML_Project/Covid_XRay_Dataset/Train"
VAL_PATH = "C:/Users/Sagar Nath/Desktop/ML_Project/Covid_XRay_Dataset/Test"

In [ ]:
#Training model
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='SGD',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
#Getting parameters
#model.summary()

In [ ]:
#Moulding train images
train_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2,zoom_range = 0.2, horizontal_flip = True)
test_dataset = image.ImageDataGenerator(rescale=1./255)

In [ ]:
#Reshaping test and validation images
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')
validation_generator = test_dataset.flow_from_directory(
    VAL_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 224 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [ ]:
#Training the model
hist_new = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2/8 [======>.......................] - ETA: 31:32 - loss: 0.7008 - acc: 0.5469

In [ ]:
#Getting summary
#summary=hist_new.history
#print(summary)

In [ ]:
model.save("model_covid_image.h5")

In [ ]:
model.evaluate_generator(train_generator,steps=1)

In [ ]:
#print(model.evaluate_generator(validation_generator,steps=1))

In [ ]:
import os
train_generator.class_indices

In [ ]:
y_actual, y_test = [],[]

In [ ]:
for i in os.listdir("C:/Users/Sagar Nath/Desktop/ML_Project/Covid_XRay_Dataset/Test/Normal/"):
    img=image.load_img("C:/Users/Sagar Nath/Desktop/ML_Project/Covid_XRay_Dataset/Test/Normal/"+i,target_size=(224,224))
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    pred=model.predict_classes(img)
    y_test.append(pred[0,0])
    y_actual.append(1)
for i in os.listdir("C:/Users/Sagar Nath/Desktop/ML_Project/Covid_XRay_Dataset/Test/Covid/"):
    img=image.load_img("C:/Users/Sagar Nath/Desktop/ML_Project/Covid_XRay_Dataset/Test/Covid/"+i,target_size=(224,224))
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    pred=model.predict_classes(img)
    y_test.append(pred[0,0])
    y_actual.append(0)

In [ ]:
y_actual=np.array(y_actual)
y_test=np.array(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cn=confusion_matrix(y_actual,y_test)

In [ ]:
sns.heatmap(cn,cmap="plasma",annot=True) #0: Covid ; 1: Normal

In [ ]:
#Using csv dataset
import pandas as pd
dataset=pd.read_csv('metadata.csv')
dataset=dataset.drop(['patientid','view','modality','date','location','folder','filename','doi','url','license',
                      'clinical_notes','other_notes','intubation_present','intubated','needed_supplemental_O2','extubated'],axis=1)

dataset['offset']=dataset['offset'].fillna(5)
dataset['sex']=dataset['sex'].fillna('M')
dataset['age']=dataset['age'].fillna(dataset['age'].mean())
dataset['temperature']=dataset['temperature'].fillna(dataset['temperature'].mean())
dataset['pO2_saturation']=dataset['pO2_saturation'].fillna(dataset['pO2_saturation'].mean())
dataset['leukocyte_count']=dataset['leukocyte_count'].fillna(dataset['leukocyte_count'].mean())
dataset['neutrophil_count']=dataset['neutrophil_count'].fillna(dataset['neutrophil_count'].mean())
dataset['lymphocyte_count']=dataset['lymphocyte_count'].fillna(dataset['lymphocyte_count'].mean())
dataset['RT_PCR_positive']=dataset['RT_PCR_positive'].fillna('Y')

dataset.loc[(dataset.finding == 'Pneumonia/Viral/COVID-19'),'finding']=1
dataset.loc[(dataset.finding != 1),'finding']=0

#dataset = dataset[dataset['RT_PCR_positive'].notna()]
dataset = dataset[dataset['survival'].notna()]
dataset = dataset[dataset['went_icu'].notna()]
dataset = dataset[dataset['in_icu'].notna()]

dataset['sex']=dataset['sex'].map({'M':1,'F':0})
dataset['RT_PCR_positive']=dataset['RT_PCR_positive'].map({'Y':1,'Unclear':0})
dataset['survival']=dataset['survival'].map({'Y':1,'N':0})
dataset['went_icu']=dataset['went_icu'].map({'Y':1,'N':0})
dataset['in_icu']=dataset['in_icu'].map({'Y':1,'N':0})


#print(dataset.head())

x=dataset.iloc[:,4:13]
y=dataset.iloc[:,3]
print(y.head())

from sklearn import model_selection as ms
x_train,x_test,y_train,y_test = ms.train_test_split(x,y,test_size = 0.20)

#Random Forest Classification
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=0)
classifier.fit(x_train,y_train)

#Predicting training set results
y_pred_train=classifier.predict(x_train)

#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_train=confusion_matrix(y_train,y_pred_train)

import seaborn as sns
sns.heatmap(cm_train,cmap="plasma",annot=True) #0: Covid ; 1: Normal

#Predicting test set results
y_pred_test=classifier.predict(x_test)

#Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm_test=confusion_matrix(y_test,y_pred_test)

import seaborn as sns
#sns.heatmap(cm_test,cmap="plasma",annot=True) #0: Covid ; 1: Normal




#dataset = dataset[dataset['intubated'].notna()]
#print(len(dataset))
#print(dataset['extubated'].value_counts())
#print(y_train.isnull().sum())



In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(x_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(x_train, y_train)))

    gb_clf2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=0)
gb_clf2.fit(x_train, y_train)
predictions = gb_clf2.predict(x_test)

print("Confusion Matrix:")
sns.heatmap(cm_train,cmap="plasma",annot=True) #0: Covid ; 1: Normal

print("Classification Report")
print(classification_report(y_test, predictions))